In [4]:
import boto3
import json

# Carregar configurações do arquivo JSON
with open('config.json') as f:
    config = json.load(f)

# Configurar o cliente S3 usando as configurações do arquivo JSON
s3 = boto3.client('s3',
                  endpoint_url=config['aws_endpoint'],
                  aws_access_key_id=config['aws_access_key_id'],
                  aws_secret_access_key=config['aws_secret_access_key'],
                  region_name='us-east-1',  # Ou a região apropriada
                  )

# Nome do bucket onde será criada a "pasta"
bucket_name = 'raw'

# Nome da "pasta" que deseja criar
folder_name = 'teste'

# Adicionar um objeto vazio que representará a "pasta" no S3
# A "pasta" será representada por um objeto sem conteúdo, apenas o nome do objeto
# No S3, um objeto com a key 'raw/' indicará a "pasta" chamada 'raw'
s3.put_object(Bucket=bucket_name, Key=folder_name)

print(f'Pasta {folder_name} criada no bucket {bucket_name}')


Pasta teste criada no bucket raw


In [9]:
import pandas as pd
import boto3
import json
import os

# Dados de exemplo para o DataFrame
data = {
    'Nome': ['João', 'Maria', 'José'],
    'Idade': [30, 25, 40],
    'Cidade': ['São Paulo', 'Rio de Janeiro', 'Belo Horizonte']
}

# Criar DataFrame
df = pd.DataFrame(data)

# Nome do arquivo CSV temporário local
temp_csv_file = 'temp_data.csv'

# Salvar DataFrame como CSV localmente
df.to_csv(temp_csv_file, index=False)

# Carregar configurações do arquivo JSON
with open('config.json') as f:
    config = json.load(f)

# Configurar o cliente S3 usando as configurações do arquivo JSON
s3 = boto3.client('s3',
                  endpoint_url=config['aws_endpoint'],
                  aws_access_key_id=config['aws_access_key_id'],
                  aws_secret_access_key=config['aws_secret_access_key'],
                  region_name='us-east-1'  # Ou a região apropriada
                  )

# Nome do bucket onde você deseja salvar o arquivo
bucket_name = 'raw'

# Caminho dentro do bucket onde você deseja salvar o arquivo
folder_name = 'base'

# Nome do arquivo no S3
s3_file_name = folder_name + 'data.csv'

# Fazer upload do arquivo CSV para o S3
s3.upload_file(temp_csv_file, bucket_name, s3_file_name)

print(f'Arquivo {s3_file_name} salvo no bucket {bucket_name}')

# Remover arquivo CSV temporário local
os.remove(temp_csv_file)


Arquivo basedata.csv salvo no bucket raw


In [13]:
import json
from pyspark.sql import SparkSession

# Carregar configurações do arquivo JSON
with open('config.json') as f:
    config = json.load(f)

# Configuração do SparkSession com recursos otimizados
spark = SparkSession.builder \
    .appName("PySpark S3 Example") \
    .config("spark.hadoop.fs.s3a.endpoint", config['aws_endpoint']) \
    .config("spark.hadoop.fs.s3a.access.key", config['aws_access_key_id']) \
    .config("spark.hadoop.fs.s3a.secret.key", config['aws_secret_access_key']) \
    .config("spark.executor.memory", "2g") \
    .config("spark.executor.cores", 2) \
    .config("spark.logConf", "true") \
    .config("spark.logLevel", "ERROR") \
    .getOrCreate()

# Exemplo de leitura de DataFrame a partir de arquivo CSV no S3
df = spark.read.csv("s3a://{}/raw/basedata.csv".format(config['raw']), header=True)

# Exibir os primeiros registros do DataFrame
df.show()

# Encerrar SparkSession
spark.stop()


KeyboardInterrupt: 